In [83]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [84]:
import json
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy

import numpy as np
import pandas as pd

In [85]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
input = '/content/drive/MyDrive/fyp/fyp2/cleaned data/manual_label_Gold_cleaned.csv'
model_name = "svalabs/twitter-xlm-roberta-bitcoin-sentiment"

In [99]:
df_ = pd.read_csv(input)
display(df_)

,Unnamed: 0,Unnamed: 0.1,index,date,tweet,is_retweet,real_score
0,0,0,0,2021-11-18 19:52:08,🔥🔥🔥 ANOTHER MASSIVE WIN! 🔥🔥🔥\n\nLUNA/USD Take-...,False,1.00
1,1,1,1,2021-10-23 09:37:54,&gt;There is no inflation\n&gt;There is some i...,False,0.00
2,2,2,2,2022-04-19 04:12:41,China’s largest broadcaster’s NFT series becom...,False,0.00
3,3,3,3,2021-05-29 22:54:12,❗Bee Network-Cloud Mining on Your Phone ❗☁️⛏️💰...,False,0.00
4,4,4,4,2022-03-18 21:04:16,"Have you heard of $PIT, one of the largest com...",False,0.25
...,...,...,...,...,...,...,...
293,295,295,295,2022-01-12 09:21:10,NFT news.\n\nKakao Entertainment to expand its...,False,0.00
294,296,296,296,2021-04-24 12:24:23,Crypto mining as it should be! \n\nZionodes pr...,False,0.00
295,297,297,297,2021-07-21 04:19:31,"The state now has 1,350 active locally-acquire...",False,0.00
296,298,298,298,2021-10-19 17:25:28,What’s amazing it’s not just for the rich and ...,False,0.25


In [100]:
training_size = 240
df = df_.copy()

sentences = df['tweet'].copy().to_list()
labels = []
urls = []



for score in df['real_score']:
  if score > 0:
    label = 'Positive'
  elif score == 0:
    label = 'Neutral'
  elif score < 0:
    label = 'Negative'
  labels.append(label)

df['label'] = labels
display(df)

training_sentences = sentences[0:training_size]
validation_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
validation_labels = labels[training_size:]

,Unnamed: 0,Unnamed: 0.1,index,date,tweet,is_retweet,real_score,label
0,0,0,0,2021-11-18 19:52:08,🔥🔥🔥 ANOTHER MASSIVE WIN! 🔥🔥🔥\n\nLUNA/USD Take-...,False,1.00,Positive
1,1,1,1,2021-10-23 09:37:54,&gt;There is no inflation\n&gt;There is some i...,False,0.00,Neutral
2,2,2,2,2022-04-19 04:12:41,China’s largest broadcaster’s NFT series becom...,False,0.00,Neutral
3,3,3,3,2021-05-29 22:54:12,❗Bee Network-Cloud Mining on Your Phone ❗☁️⛏️💰...,False,0.00,Neutral
4,4,4,4,2022-03-18 21:04:16,"Have you heard of $PIT, one of the largest com...",False,0.25,Positive
...,...,...,...,...,...,...,...,...
293,295,295,295,2022-01-12 09:21:10,NFT news.\n\nKakao Entertainment to expand its...,False,0.00,Neutral
294,296,296,296,2021-04-24 12:24:23,Crypto mining as it should be! \n\nZionodes pr...,False,0.00,Neutral
295,297,297,297,2021-07-21 04:19:31,"The state now has 1,350 active locally-acquire...",False,0.00,Neutral
296,298,298,298,2021-10-19 17:25:28,What’s amazing it’s not just for the rich and ...,False,0.25,Positive


In [89]:
print(len(training_sentences))
print(len(validation_sentences))
print(type(training_sentences))

240
58
<class 'list'>


In [90]:

#model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [91]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True,
                            return_tensors='tf')
val_encodings = tokenizer(validation_sentences,
                          truncation=True,
                          padding=True,
                          return_tensors='tf')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [92]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

In [93]:
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3,from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFXLMRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.


In [94]:
test_sentence = "bitcoin is good "
test_sentence_sarcasm = "Amphibious pitcher makes debut"

predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
tf_output = model.predict(predict_input)[0]
tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
print(tf_prediction)

predict_input_s = tokenizer.encode(test_sentence_sarcasm,
                                   truncation=True,
                                   padding=True,
                                   return_tensors="tf")
tf_output_s = model.predict(predict_input_s)[0]
tf_prediction_s = tf.nn.softmax(tf_output_s, axis=1).numpy()[0]
print(tf_prediction_s)

[0.00496938 0.1390723  0.8559583 ]
[0.00405704 0.10589628 0.89004666]


In [95]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [98]:
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=3,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/3


UnimplementedError: ignored

In [ ]:
model.save_pretrained("/tmp/sentiment_custom_model")

In [ ]:
#### Load saved model and run predict function

In [ ]:
loaded_model = TFAutoModelForSequenceClassification.from_pretrained("/tmp/sentiment_custom_model")

In [ ]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [ ]:
test_sentence = "Shock results and Team USA gets first gold"
test_sentence_sarcasm = "Amphibious pitcher makes debut"

predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
tf_output = loaded_model.predict(predict_input)[0]
tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
print(tf_prediction)

predict_input_s = tokenizer.encode(test_sentence_sarcasm,
                                   truncation=True,
                                   padding=True,
                                   return_tensors="tf")
tf_output_s = model.predict(predict_input_s)[0]
tf_prediction_s = tf.nn.softmax(tf_output_s, axis=1).numpy()[0]
print(tf_prediction_s)